In [51]:
from fastapi import FastAPI
from typing import List
import pandas as pd
import numpy as np
import os
import folium
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

app = FastAPI()

# Path to the directory containing your cleaned CSV files
DATA_DIR = '/Users/annaschneider/Desktop/clean_files'

def get_csv_files():
    return [f for f in os.listdir(DATA_DIR) if f.endswith('.csv')]

@app.get("/routes")
def list_routes():
    files = get_csv_files()
    routes = []
    for file in files:
        route = file[:7]
        if route not in routes:
            routes.append(route)
    return {"routes": routes}

@app.get("/routes/{route}/dates")
def list_dates(route: str):
    files = get_csv_files()
    dates = []
    for file in files:
        if file.startswith(route):
            date = file[8:-4]  # Extract the date from the filename
            dates.append(date)
    return {"dates": dates}


@app.get("/routes/{route}/dates/{date}/map")
def map_route(route: str, date: str):
    file_path = os.path.join(DATA_DIR, f"{route}_{date}.csv")
    if not os.path.exists(file_path):
        return {"error": "file not found"}
    
    df = pd.read_csv(file_path)
    flight_coords = []
    
    def add_flight_path(df, flight_key):
        flight_data = df[df['fltKey'] == flight_key]
        coordinates = flight_data[['Latitude', 'Longitude']].values.tolist()
        flight_coords.append(coordinates)
    
    flight_keys = df['fltKey'].unique()
    for key in flight_keys:
        add_flight_path(df, key)
    
    return flight_coords

@app.get("/routes/{route}/dates/{date}/heatmap")
def heatmap_route(route: str, date: str):
    file_path = os.path.join(DATA_DIR, f"{route}_{date}.csv")
    if not os.path.exists(file_path):
        return {"error": "file not found"}
    
    df = pd.read_csv(file_path)
    heat_data = df[['Latitude', 'Longitude']].values.tolist()
    
    return heat_data

@app.get("/routes/{route}/dates/{date}/prediction")
def route_prediction(route: str, date: str):
    file_path = os.path.join(DATA_DIR, f"{route}_{date}.csv")
    data = pd.read_csv(file_path)

    flight_keys = data['fltKey'].unique()  # Change test size if wanted
    train_keys, test_keys = train_test_split(flight_keys, test_size=0.2, random_state=42)

    train_data = data[data['fltKey'].isin(train_keys)]
    test_data = data[data['fltKey'].isin(test_keys)]
    len(train_keys), len(test_keys)

    scaler = MinMaxScaler()
    data[['Latitude', 'Longitude']] = scaler.fit_transform(data[['Latitude', 'Longitude']])

    # Sequence length controls number of points used to predict the next point
    seq_length = 1000

    def create_knn_sequences(data, seq_length):
        sequences = []
        targets = []
        for fltKey in data['fltKey'].unique():
            flight_data = data[data['fltKey'] == fltKey]
            coords = flight_data[['Latitude', 'Longitude']].values
            if len(coords) > seq_length:
                for i in range(len(coords) - seq_length):
                    sequences.append(coords[i:i+seq_length])
                    targets.append(coords[i+seq_length])
        return np.array(sequences), np.array(targets)

    X_train_knn, y_train_knn = create_knn_sequences(train_data, seq_length)

    knn_model = KNeighborsRegressor(n_neighbors=5)
    knn_model.fit(X_train_knn.reshape(X_train_knn.shape[0], -1), y_train_knn)
    
    # num_flights is the number of flights printed
    def plot_paths(test_data, knn_model, scaler, num_flights=3, seq_length=1000):
        fig, axes = plt.subplots(num_flights, 1, figsize=(10, 5 * num_flights))
    
        test_flights = test_data['fltKey'].unique()[:num_flights]
        for i, fltKey in enumerate(test_flights):
            flight_data = test_data[test_data['fltKey'] == fltKey]
            actual_coords = flight_data[['Latitude', 'Longitude']].values
            initial_coords = actual_coords[:seq_length]

            # Predict subsequent points based on the initial 400 coordinates
            predicted_coords = []
            current_sequence = initial_coords.copy()
        
            for j in range(seq_length, len(actual_coords)):
                pred = knn_model.predict(current_sequence.reshape(1, -1))
                predicted_coords.append(pred[0])
                current_sequence = np.vstack([current_sequence[1:], pred])
        
            actual_coords_rescaled = scaler.inverse_transform(actual_coords)
            initial_coords_rescaled = actual_coords_rescaled[:seq_length]
            predicted_coords_rescaled = scaler.inverse_transform(predicted_coords)

    plot_paths(test_data, knn_model, scaler, num_flights=3, seq_length=1000)

    return actual_coords_rescaled, initial_coords_rescaled, predicted_coords_rescaled 

In [53]:
import uvicorn
from threading import Thread

def run_app():
    uvicorn.run(app, host="127.0.0.1", port=8006, log_level="info")

# Start the FastAPI app in a separate thread
app_thread = Thread(target=run_app, daemon=True)
app_thread.start()


INFO:     Started server process [64228]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8006 (Press CTRL+C to quit)


In [55]:
import httpx

# Test the /routes endpoint
response = httpx.get("http://127.0.0.1:8006/routes")
response.json()

INFO:     127.0.0.1:64778 - "GET /routes HTTP/1.1" 200 OK


{'routes': ['SFO_SYR', 'SFO_IAD']}

In [57]:
response = httpx.get("http://127.0.0.1:8006/routes/SFO_IAD/dates")
response.json()

INFO:     127.0.0.1:64784 - "GET /routes/SFO_IAD/dates HTTP/1.1" 200 OK


{'dates': ['20230201_20230208', '20230203', '20230201_20230203']}

In [59]:
response = httpx.get("http://127.0.0.1:8006/routes/SFO_IAD/dates/20230203/map")
print(response.json())

INFO:     127.0.0.1:64785 - "GET /routes/SFO_IAD/dates/20230203/map HTTP/1.1" 200 OK
[[[37.61924, -122.38366], [37.6192, -122.38356], [37.61918, -122.38355], [37.61916, -122.38355], [37.61914, -122.38354], [37.61914, -122.38352], [37.61916, -122.38348], [37.61915, -122.38347], [37.61915, -122.38346], [37.61916, -122.3834], [37.61915, -122.38338], [37.61913, -122.38336], [37.61913, -122.38331], [37.61913, -122.38329], [37.61914, -122.38328], [37.61914, -122.38324], [37.61916, -122.38319], [37.61917, -122.38316], [37.61919, -122.38314], [37.61918, -122.38312], [37.61919, -122.38309], [37.6192, -122.38306], [37.6192, -122.38303], [37.61921, -122.38301], [37.6192, -122.38298], [37.6192, -122.38296], [37.6192, -122.38294], [37.61919, -122.38291], [37.61918, -122.38289], [37.61916, -122.38286], [37.61915, -122.38284], [37.61914, -122.38282], [37.61915, -122.3828], [37.61916, -122.3828], [37.61917, -122.38279], [37.61918, -122.38279], [37.61919, -122.38278], [37.6192, -122.38277], [37.61921, 

In [61]:
response = httpx.get("http://127.0.0.1:8006/routes/SFO_IAD/dates/20230203/heatmap")
print(response.json())

INFO:     127.0.0.1:64799 - "GET /routes/SFO_IAD/dates/20230203/heatmap HTTP/1.1" 200 OK
[[37.61924, -122.38366], [37.6192, -122.38356], [37.61918, -122.38355], [37.61916, -122.38355], [37.61914, -122.38354], [37.61914, -122.38352], [37.61916, -122.38348], [37.61915, -122.38347], [37.61915, -122.38346], [37.61916, -122.3834], [37.61915, -122.38338], [37.61913, -122.38336], [37.61913, -122.38331], [37.61913, -122.38329], [37.61914, -122.38328], [37.61914, -122.38324], [37.61916, -122.38319], [37.61917, -122.38316], [37.61919, -122.38314], [37.61918, -122.38312], [37.61919, -122.38309], [37.6192, -122.38306], [37.6192, -122.38303], [37.61921, -122.38301], [37.6192, -122.38298], [37.6192, -122.38296], [37.6192, -122.38294], [37.61919, -122.38291], [37.61918, -122.38289], [37.61916, -122.38286], [37.61915, -122.38284], [37.61914, -122.38282], [37.61915, -122.3828], [37.61916, -122.3828], [37.61917, -122.38279], [37.61918, -122.38279], [37.61919, -122.38278], [37.6192, -122.38277], [37.6192

In [ ]:
response = httpx.get("http://127.0.0.1:8006/routes/SFO_IAD/dates/20230201_20230208/prediction", timeout = 120.0)
print(response.json())